In [2]:
from sklearn.model_selection import train_test_split
from src import data_preprocessing as preproc

csv_path = r'data\raw\v2\pynq_1_data.csv'
X, y = preproc.preprocess_csv(csv_path, 0)['puf_response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [3]:
import keras
from keras import layers

In [4]:
model = keras.Sequential(
    [
        keras.Input(shape=(130,)),
        layers.Dense(256, activation="relu", name="hidden_layer_1",),
        layers.Dense(256, activation="relu", name="hidden_layer_2"),
        layers.Dense(128, activation="sigmoid", name="output_layer"),
    ]
)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# model.summary()

In [5]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0000e+00 - loss: 0.0562 - val_accuracy: 0.0000e+00 - val_loss: 3.7269e-07
Epoch 2/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.0000e+00 - loss: 2.3774e-07 - val_accuracy: 0.0000e+00 - val_loss: 6.1318e-08
Epoch 3/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 4.7342e-08 - val_accuracy: 0.0000e+00 - val_loss: 2.3714e-08
Epoch 4/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 2.0548e-08 - val_accuracy: 0.0000e+00 - val_loss: 1.4255e-08
Epoch 5/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 1.3079e-08 - val_accuracy: 0.0000e+00 - val_loss: 1.0297e-08
Epoch 6/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 9.6626e-09 - val_accuracy: 0.0000e+00 - val_loss: 8.1367e-09
Epoch 7/50
550/550 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0000e+00 - loss: 7.7310e-09 - val_accuracy: 0.0000e+00 - val_loss: 6.7733e

In [ ]:
import numpy as np

y_pred_probs = model.predict(X_test)

y_pred = (y_pred_probs > 0.5).astype("int")

per_bit_accuracy = np.mean(y_pred == y_test)
print(f"Per-Bit Accuracy: {per_bit_accuracy * 100:.4f}%")

bit_error_rate = 1 - per_bit_accuracy
print(f"Bit Error Rate (BER): {bit_error_rate * 100:.4f}%")

num_exact_matches = np.sum(np.all(y_pred == y_test, axis=1))
total_test_samples = len(y_test)
print(f"Total Exactly Correct Responses: {num_exact_matches} / {total_test_samples} ({num_exact_matches/total_test_samples * 100:.2f}%)")

print("\n--- Example Predictions vs. True Values ---")
# Displaying the first 5 samples from the test set
for i in range(5):
    print(f"Sample #{i+1}")
    print(f"  Predicted: {''.join(map(str, y_pred[i]))}")
    print(f"  Actual:    {''.join(map(str, y_test[i]))}")
    # Calculate errors for this specific sample
    errors = np.sum(y_pred[i] != y_test[i])
    print(f"  (Errors in this sample: {errors})")
    print("-" * 20)

138/138 ━━━━━━━━━━━━━━━━━━━━ 0s 785us/step
Per-Bit Accuracy: 100.0000%
Bit Error Rate (BER): 0.0000%
Total Exactly Correct Responses: 4400 / 4400 (100.00%)

--- Example Predictions vs. True Values ---
Sample #1
  Predicted: 11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  Actual:    11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  (Errors in this sample: 0)
--------------------
Sample #2
  Predicted: 11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  Actual:    11001010101111000100100101000000000100000101001100100000101100010110000111101000000101000000011100000000000110001100100001100011
  (Errors in this sample: 0)
--------------------
Sample #3
  Predicted: 1100101010111100010010010100000000010000010100110010000010110001011000011110100000010100

In [ ]:
import numpy as np

# Since every row in y_test is the ideal response, we can just grab the first one.
ideal_response_bits = y_test[0]

print("--- Running Experiment 1: Zero Vector Input ---")

zero_input = np.zeros((1, 130))

zero_pred_probs = model.predict(zero_input)
zero_pred_bits = (zero_pred_probs > 0.5).astype(int)[0]

print(ideal_response_bits)
print(zero_pred_bits)

--- Running Experiment 1: Zero Vector Input ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
[1 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1
 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1]
[1 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1
 1 0 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 1 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1]
